- [ ] Ler todos os arquivos
- [ ] Separar por municipios
- [ ] Aplicar funcao de limpeza
- [ ] Salvar no banco

In [1]:
PASTA = 'dados'

# Ler Arquivo

In [2]:
import pandas as pd

df = pd.read_parquet(f'{PASTA}/0.raw.parquet')

# Funçao suavizar

In [3]:
def suavizar(df, window_size=3, threshold=2):
    """
    Smooth data by replacing outliers with previous day's casosNovos.
    municipio, estado must be unique.
    
    Parameters:
    - df: pandas DataFrame with columns 'date' and 'casosNovos'
    - window_size: number of days to consider in rolling window
    - threshold: number of standard deviations to use for outlier detection
    
    Returns:
    - DataFrame with smoothed values in the column 'novos_casos_novos'
    """
    # Make a copy to avoid modifying original data
    df_smoothed = df.copy()

    df['novos_casos_novos'] = df['casosNovos']
    
    # Calculate rolling statistics
    rolling_mean = df['novos_casos_novos'].rolling(window=window_size, center=True, min_periods=1).mean()
    rolling_std = df['novos_casos_novos'].rolling(window=window_size, center=True, min_periods=1).std()
    
    # Identify outliers (values outside mean ± threshold*std)
    lower_bound = rolling_mean - threshold * rolling_std
    upper_bound = rolling_mean + threshold * rolling_std
    
    is_outlier = (df['novos_casos_novos'] < lower_bound) | (df['novos_casos_novos'] > upper_bound)
    
    # Replace outliers with previous day's value
    df_smoothed['novos_casos_novos'] = df['novos_casos_novos'].where(~is_outlier, df['novos_casos_novos'].shift(1))
    df_smoothed['novos_casos_novos'] = df['novos_casos_novos'].where(~is_outlier, df['novos_casos_novos'].shift(1))
    
    # For the first row (no previous value), use the next value if available
    if is_outlier.iloc[0] and len(df) > 1:
        df_smoothed.iloc[0, df_smoothed.columns.get_loc('novos_casos_novos')] = df['novos_casos_novos'].iloc[1]
    
    return df_smoothed

# Recalcula casos acumulados

In [4]:
def recalcula_casos_acumulados(df):
  df['novos_casos_acumulados'] = df['casosNovos'].cumsum()
  return df

# Função de limpeza

In [5]:
def limpar(df):
  df = suavizar(df)
  df = recalcula_casos_acumulados(df)
  return df

# Função principal - sincrona e csv

In [6]:
import logging

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
)

In [7]:
from tqdm import tqdm

def rodar(df):
    logging.info("Processando municipios_unicos")
    municipios_unicos = set(zip(df['municipio'], df['estado']))
    logging.info("Processamento concluído")
    
    logging.info("Filtrando o dataframe ")
    mascara = df.apply(lambda x: (x['municipio'], x['estado']) in municipios_unicos, axis=1)
    filtrado_df = df[mascara]
    logging.info("Processamento concluído")
    
    logging.info("Processando grupos")
    agrupado = filtrado_df.groupby(['municipio', 'estado'])
    logging.info("Processamento concluído")

    resultado_df = []
    
    for (municipio, estado), group_df in tqdm(agrupado, desc="Processando"):
        try:
            resultado_df.append(limpar(group_df))
        except Exception as e:
            logging.error(f"{estado}_{municipio} não foi salvo")
            logging.error(e)
    
    resultado_df = pd.concat(resultado_df)
    return resultado_df

In [8]:
processado_df = rodar(df)

2025-05-12 10:28:21 - INFO - Processando municipios_unicos
2025-05-12 10:28:24 - INFO - Processamento concluído
2025-05-12 10:28:24 - INFO - Filtrando o dataframe 
2025-05-12 10:29:31 - INFO - Processamento concluído
2025-05-12 10:29:31 - INFO - Processando grupos
2025-05-12 10:29:31 - INFO - Processamento concluído
Processando:  99%|█████████▉| 5570/5598 [00:13<00:00, 399.53it/s]


In [9]:
processado_df.to_parquet(f'{PASTA}/1.limpo.parquet', index=False)